# Do it yourself

In the following notebook, you'll be exploring how you can load and visualise data using a set of simple Open Data Cube commands.

Before you get started, you should save this notebook to somewhere outside the `examples` folder, so that your work wont get overwritten when you next log in.

As you work through this notebook you will:
1. Pick a study area anywhere in Australia.
1. Explore available data products for your study area.
1. Set up a datacube load command to load data for your study area.
1. Plot data that has been loaded, exploring plotting of different timesteps.
1. Export data to an image format to view on your local computer.

Let's get started.

First, please use Google Maps to pick a study site in Australia and click on the map. You should be able to select the lat/lon coordiantes. Visit [maps.google.com](https://www.google.com/maps/@-28.6035447,135.9291226,4.93z) to select a site. The image below shows the coordinates that you can select at the bottom of the image.

![google maps coordinates](images/google_maps.jpg "Google Maps Coordinates")


*Note that there are three example study sites below that you can use if you like. Uncomment the one you'd like to look at. Once you've picked a site, you need to run or re-run each cell below to run the analysis on the new data.*

In [ ]:
# Example study site one, Dead Dog Creek in Queensland
coordinates = [-14.642744, 144.899747]

# Example study site two, Giles Creek near Alice Springs
# coordinates = [-23.765165, 134.724024]

# Example study site three, Lake Disappointment in WA
# coordinates = [-23.481127, 122.817712]

# Paste your coordinates here, and remove the hash to uncomment
# coordinates = []

In [ ]:
# For simplicity, store the centre X and Y coords
centre_x = coordinates[1]  # longitude
centre_y = coordinates[0]  # latitude

# And now make a bounding box with them
# Note that you can change the box_size to load more data. More data means longer load, though!
box_size = 0.1
bounding_box_x =  (centre_x - box_size, centre_x + box_size)
bounding_box_y =  (centre_y - box_size, centre_y + box_size)

## Loading a cube of data

This next section does the load. First we set up a datacube object, `dc`, which has all the functions of the Open Data Cube library, and then we use the `dc.load()` function to load data. You can see the parameters that we've set below, but you can change them, for example, adding or removing measurements based on [product metadata](ODC_and_DEA_Metadata.ipynb).

In [ ]:
import datacube
import warnings
warnings.filterwarnings('ignore')  # suppress warnings

dc = datacube.Datacube(app='do-it-yourself')

# This command here does the loading of data
# Please be patient, it can take some time to load, depending on the size of your study area
# For the example study area, this took 1 minute
data_cube = dc.load(
    product='s2a_nrt_granule',
    x=bounding_box_x,
    y=bounding_box_y,
    resolution = (-10, 10),
    output_crs='epsg:3577',
    measurements=(
        'nbar_red',
        'nbar_green',
        'nbar_blue',
        'nbar_nir_1'
    )
)

In [ ]:
# This will give information on how much data was loaded
# Most interesting is the 'Dimensions' section, that tells you how many timesteps were loaded
# and the x/y resolution of the cube.
print(data_cube)

## Plotting data

The next step uses Matplotlib to plot some data. We use a quick function to prepare the data, called `three_band_image`, which will prepare the measurements from the datacube we loaded into something Matplotlib expects. Later, we show another way to plot a simple single-band image.

You should experiment with plotting different timesteps or combinations of bands.

In [ ]:
import matplotlib.pyplot as plt

# Note that this import should work if this notebook is either in examples or in the root folder.
try:
    from utils.utils import three_band_image
except ImportError:
    from examples.utils.utils import three_band_image


# Change these!
# You can change the time to anything from 0 to one less than the number of timesteps.
# For example, if there are 6 timesteps, the largest time you can use is 5.
time = 1

# The bands can be any of the bands that we loaded above, so any of:
# 'nbar_red', 'nbar_green', 'nbar_blue', 'nbar_nir_1'
# You can experiment with plotting in false-colour, for example, try ['nbar_nir_1', 'nbar_red', 'nbar_green']
# You can switch between the two band lists specified below or write your own.
bands = ['nbar_red', 'nbar_green', 'nbar_blue']
# bands = ['nbar_nir_1', 'nbar_red', 'nbar_green']


# Prepare the image
img_toshow = three_band_image(
    data_cube,
    bands=bands,
    time=time
)
# Format the time string nicely
time_string = str(data_cube.time.isel(time=time).values).split('.')[0]

# And plot it
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_title("Timestep {}".format(time_string), fontweight='bold', fontsize=16)
ax.set_xticklabels(data_cube.x.values)
ax.set_yticklabels(data_cube.y.values)
ax.set_xlabel('Easting', fontweight='bold')
ax.set_ylabel('Northing', fontweight='bold')

plt.imshow(img_toshow)

##  Exporting data
The last task here is to export the data for your study site. You can change the name of the filename so that you know what the file is going to be called. After the file has been created, you can download it from the Jupyter directory it was exported into.

In [ ]:
from datacube import helpers

# You can change this, if you like.
filename = "example.tiff"

helpers.write_geotiff(dataset=data_cube.isel(time=1), filename=filename)

## Stretch goal: Calculate NDVI

If you've come this far and you'd like to do something a bit fancier, you can have a go at calculating the normalised difference vegetation index (NDVI) over your study site. There is a definition of what [NDVI is on Wikipedia](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index).

Basically, you need to use the following formula:

$$
\begin{aligned}
ndvi & = \frac{(nir - red)}{(nir + red)} \\
\end{aligned}
$$
    
Some hints:
 * You can access bands of an Xarray (the data format we're using) with their name, like this: `data_cube.bandname`
 * You can do simple math with bands by simply referring to them, like this: `data_cube.bandname_1 + data_cube.bandname_2`
 * The two band names you're after are `nbar_nir_1`, which is near infra-red, and `nbar_red`, which is red.


In [ ]:
# Calculate NDVI here
ndvi =  # Your calculation goes here

# This is the simple way to plot
# Note that high values are likely to be vegetation.
plt.figure(figsize=(10,10))
ndvi.isel(time=1).plot()